In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='your_api_key',
    model_name="llama-3.1-70b-versatile"
)

res = llm.invoke("The first man to land on moon was ...")
print(res.content)

The first man to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37315")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Retail Associate, SEAS - Cabazon

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate, SEAS - CabazonCabazon, CaliforniaBecome part of the Converse Te

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [25]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail Associate, SEAS - Cabazon',
  'experience': 'No specific experience mentioned',
  'skills': ['Positive attitude',
   'Knowledge of footwear, apparel and accessories',
   'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time',
   'Ability to work nights, weekends and holidays based on store needs'],
  'description': 'Converse is looking for a Retail Associate to join their team in Cabazon, California. The ideal candidate will have a positive attitude, be knowledgeable about footwear, apparel and accessories, and be able to work in a fast-paced environment. The starting hourly rate for this position is $17.50/hour.'}]

In [26]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [27]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [39]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [41]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [36]:
job

[{'role': 'Retail Associate, SEAS - Cabazon',
  'experience': 'No specific experience mentioned',
  'skills': ['Positive attitude',
   'Knowledge of footwear, apparel and accessories',
   'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time',
   'Ability to work nights, weekends and holidays based on store needs'],
  'description': 'Converse is looking for a Retail Associate to join their team in Cabazon, California. The ideal candidate will have a positive attitude, be knowledgeable about footwear, apparel and accessories, and be able to work in a fast-paced environment. The starting hourly rate for this position is $17.50/hour.'}]

In [37]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Operations with AtliQ's AI-Powered Solutions

Dear Hiring Manager,

I came across the Retail Associate role at Converse in Cabazon, California, and I was impressed by the emphasis on creating a seamless customer experience. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI and software consulting services that can help elevate your retail operations to the next level.

At AtliQ, we specialize in developing tailored solutions that cater to the unique needs of businesses like yours. Our expertise in process automation, data analytics, and machine learning can help you optimize store operations, streamline inventory management, and enhance customer engagement.

Our team has a proven track record of delivering scalable and efficient solutions that drive business growth. I'd like to highlight a few examples from our portfolio:

* Our machine learning-powered solution for inventory management can help you optimize stock levels, red